In [1]:
# import packages 
import pandas as pd
import numpy as np
import matplotlib
import glob
import json
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score
from sklearn.svm import SVC

RSEED = 50

In [2]:
def load_kickstarter_data(datapath):
    '''datapath = location of csv files to be loaded'''
    # List with the names of all the csv files in the path
    csv_files = glob.glob(datapath+'/*.csv')

    print(f'Total files: {len(csv_files)}')

    # Loop through the files
    for file_idx, csv_file in enumerate(csv_files): 
        # create dataframe from 1st csv       
        if file_idx == 0:
            df_ks = pd.read_csv(csv_file)
            print(f'File number {file_idx + 1} added to dataframe')
        else:
            # create dataframe from idx csv
            df = pd.read_csv(csv_file)
            # check files are all in same
            if  np.all(df.columns == df_ks.columns) == False:
                print(f'Column format of {csv_file} does not match {csv_files[0]}. Please check and try again')
                return
            else:
                # append to initial dataframe                   
                df_ks = pd.concat([df_ks, df], axis=0, ignore_index=True)       
                print(f'File number {file_idx + 1} added to dataframe')
    print('File import done')
    return df_ks

In [3]:
def load_kickstarter_data_short(datapath):
    ''' 
    This is a version of the main function to load jsut 2 files for use in testing
    datapath = location of csv files to be loaded
    '''
    # List with the names of all the csv files in the path
    csv_files = glob.glob(datapath+'/*.csv')

    print(f'Total files: {len(csv_files)}')

    # Loop through the files
    for file_idx, csv_file in enumerate(csv_files): 
        # create dataframe from 1st csv       
        if file_idx == 0:
            df_ks = pd.read_csv(csv_file)
            print(f'File number {file_idx + 1} added to dataframe')
        else:
            # create dataframe from idx csv
            df = pd.read_csv(csv_file)
            # check files are all in same
            if  np.all(df.columns == df_ks.columns) == False:
                print(f'Column format of {csv_file} does not match {csv_files[0]}. Please check and try again')
                return
            else:
                # append to initial dataframe                   
                df_ks = pd.concat([df_ks, df], axis=0, ignore_index=True)       
                print(f'File number {file_idx + 1} added to dataframe')
                # This is here to prevent more than 2 files being loaded to save time in testing
                break
    print('File import done')
    return df_ks

In [4]:
def extract_json_data(data):
    ''' This function extracts specific sub fields from json files embedded in columns of a dataframe
        data: dataframe containing column with json data'''
    data['category_name'] = pd.DataFrame.from_dict([json.loads(data["category"][i])['name'] for i in range(data.shape[0])])
    data['category_slug'] = pd.DataFrame([json.loads(data["category"][i])['slug'] for i in range(data.shape[0])])
    # Split slug into main category and sub category
    data[['category_main','category_sub']] = df.category_slug.str.split(pat='/', n=1, expand=True)
    data.drop(labels = ['category','category_slug'], axis=1, inplace=True)
    
    print('json columns extracted')
    return data

In [5]:
def get_duration(data):
    #Convert from unix time stamp to more readable time format
    data['converted_deadline'] = pd.to_datetime(data['deadline'], unit='s')
    data['converted_launched_at'] = pd.to_datetime(data['launched_at'], unit='s')
    #Create project duration variable
    data['project_duration_days'] = (data['converted_deadline'] - data['converted_launched_at']).dt.days
    # Drop redundant columns
    data.drop(columns=['deadline', 'launched_at'], inplace=True)
    return data

In [6]:
def get_target(data,target='state', new_target_var='success', success_label='successful'):
    '''
    creates a dummy variable out of the state to be used as dependant variable
    '''
    #data('success') = data['state'].apply(lambda x: 1 if x == 'successful' else 0)
    data[new_target_var] = data[target].apply(lambda x: 1 if x == success_label else 0)
    return data

In [7]:
def get_target_and_features(data, target_var='success'):
    '''
    Function that splits dataset into target and feature dataframes
    '''
    #target = data['success']
    target = data[target_var]
    data.drop([target_var,'state'], axis = 1, inplace=True)
    features = data

    print('target and features split is done')
    return target, features

In [8]:
def currency_conversion(data):
    # Convert the currency of all projects to USD. 
    # We use static_usd_rate since this is what was used for usd_pledged
    data['usd_goal'] = data['goal'] * data['static_usd_rate']
    # drop goal and static_usd_rate to remove redundant data
    data.drop(columns=['goal','static_usd_rate'], inplace=True)
    return data


In [9]:
def feature_engineering(data):
    '''
    Create new features: Blurb length
    '''
    data['blurb_length'] = data.blurb.apply(lambda x: len(str(x)))
    return data

In [10]:
def fix_skew(data, skewed=['usd_goal']):
    '''Log-transform the skewed features'''
    data[skewed] = data[skewed].apply(lambda x: np.log(x + 1))
    return data

In [11]:
def scale_features(data, num_columns=['usd_goal','project_duration_days','blurb_length']):
    ''' Initialize a scaler, then apply it to the features'''    
    scaler = MinMaxScaler()
    data[num_columns] = scaler.fit_transform(data[num_columns])    
    return data

In [12]:
def drop_columns(data):
    '''remove unnecessary columns'''
    # Drop due to many missing values
    data.drop(columns = ['friends', 'is_backing', 'is_starred', 'permissions'], inplace=True)
    # Some Json strings varariables with unusable or already used data
    data.drop(columns = ['creator', 'location', 'photo', 'profile', 'slug', 'urls'], inplace=True)
    # Columns that are not specific to the campaign or are redundant or are technical data unrelated to campaign
    data.drop(columns = ['created_at','currency', 'currency_symbol', 'currency_trailing_code', 
                     'current_currency', 'disable_communication',
                     'is_starrable', 'source_url', 'spotlight', 'staff_pick', 
                     'usd_type', 'state_changed_at','fx_rate'], inplace=True)
    # drop columns due to being linked to dependent variable which would not be known in advance
    data.drop(columns = ['backers_count', 'converted_pledged_amount', 'pledged', 'usd_pledged','id'], inplace=True) # to be checked 'backers_count'
    # drop columns that are not used                
    data.drop(columns = ['blurb', 'name', 'converted_deadline', 'converted_launched_at','category_name'], inplace=True) #'category_slug'
    return data

In [13]:
def test_train_split_kickstarter(features, target, test_size = 0.2, random_state = RSEED):
    '''
    Split data into train and test sets based on features and target dataframes.
    Shows results of the split and returns four dataframes
    '''
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = test_size, random_state = random_state)
    # Show the results of the split
    print ("Training set has {} samples.".format(X_train.shape[0]))
    print ("Testing set has {} samples.".format(X_test.shape[0]))
    return X_train, X_test, y_train, y_test

In [14]:
def train_using_gini(X_train, y_train):  
    # Creating the decision tree classifier object 
    clf_tree = DecisionTreeClassifier(criterion = "gini", 
            max_depth=3, min_samples_leaf=5) 
    # Performing training 
    clf_tree.fit(X_train, y_train) 
    return clf_tree

In [15]:
# Function to make predictions 
def prediction(X_test, clf_object): 
  
    # Predicton on test data with model trained using either giniIndex
    y_pred = clf_object.predict(X_test) 
    print("Predicted values:\n") 
    print(y_pred) 
    return y_pred 

In [16]:
def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: \n", 
    confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : \n", 
    accuracy_score(y_test, y_pred)*100) 
      
    print("Report : \n", 
    classification_report(y_test, y_pred)) 

In [17]:
def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size'
    start = time() # Get start time
    learner = learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set,
    #       then get predictions on the first 300 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples
    results['acc_train'] = accuracy_score(y_train[:300],predictions_train)
        
    # TODO: Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test,predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples
    results['f_train'] = fbeta_score(y_train[:300],predictions_train,beta=0.5)
        
    # TODO: Compute F-score on the test set
    results['f_test'] = fbeta_score(y_test,predictions_test,beta=0.5)
       
    # Success
    print ("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [23]:
""" MAIN SCRIPT ============================================================="""
if __name__ == '__main__':
    # this script loads data and performs all manipulations in order to get it ready for use in a model
    # Read data

    df = load_kickstarter_data('kickstarter/data')
    
    # Extract category data from json
    df = extract_json_data(df)
    
    # drop campaigns that are still ongoing
    df = df[df.state != 'live'] # change this to sucessful and 
    
    # convert unix timestamps and calculate campaign duration
    df = get_duration(df)
    
    # Get blurb length
    df = feature_engineering(df)
    
    #create goal data in single currency
    df = currency_conversion(df)
    
    # encode target variable 'state' to numerical values, success is 1 all others are fail and 0
    df = get_target(df,target='state', new_target_var='success', success_label='successful')

    # drop unnecessary columns
    df=drop_columns(df)
    df.head()
       
    # Split the data into features and target label
    target, features = get_target_and_features(df)

    # split categorical columns into dummies
    features = pd.get_dummies(features, columns=['country', 'category_main','category_sub'], drop_first=True) #Avoid dummy trap   

    # address skew    
    num_columns = ['project_duration_days', 'blurb_length', 'usd_goal']
    features = fix_skew(features, skewed=num_columns)
 
    # scale numerical features
    features = scale_features(features, num_columns)
    
     # Split into training and test set
    X_train, X_test, y_train, y_test = test_train_split_kickstarter(features, target, test_size = 0.2, random_state = RSEED)


Total files: 56
File number 1 added to dataframe
File number 2 added to dataframe
File number 3 added to dataframe
File number 4 added to dataframe
File number 5 added to dataframe
File number 6 added to dataframe
File number 7 added to dataframe
File number 8 added to dataframe
File number 9 added to dataframe
File number 10 added to dataframe
File number 11 added to dataframe
File number 12 added to dataframe
File number 13 added to dataframe
File number 14 added to dataframe
File number 15 added to dataframe
File number 16 added to dataframe
File number 17 added to dataframe
File number 18 added to dataframe
File number 19 added to dataframe
File number 20 added to dataframe
File number 21 added to dataframe
File number 22 added to dataframe
File number 23 added to dataframe
File number 24 added to dataframe
File number 25 added to dataframe
File number 26 added to dataframe
File number 27 added to dataframe
File number 28 added to dataframe
File number 29 added to dataframe
File nu

In [34]:
# Check shape of dataframes
print('X_train.shape:',X_train.shape)
print('X_test.shape:',X_test.shape)
print('y_train.shape:',y_train.shape)
print('y_test.shape:',y_test.shape)
print('y_pred.shape:',y_pred.shape)

Index(['project_duration_days', 'blurb_length', 'usd_goal', 'country_AU',
       'country_BE', 'country_CA', 'country_CH', 'country_DE', 'country_DK',
       'country_ES',
       ...
       'category_sub_wearables', 'category_sub_weaving', 'category_sub_web',
       'category_sub_webcomics', 'category_sub_webseries',
       'category_sub_woodworking', 'category_sub_workshops',
       'category_sub_world music', 'category_sub_young adult',
       'category_sub_zines'],
      dtype='object', length=181)

X_train.shape: (161528, 181)
X_test.shape: (40383, 181)
y_train.shape: (161528,)
y_test.shape: (40383,)
y_pred.shape: (40383,)


In [22]:
def simple_decsion_tree(X_train, X_test, y_train, y_test):
    '''This is the first model to set a baseline prediction'''
    # Fit a simple decision tree first
    clf_gini = train_using_gini(X_train, y_train) 
    
    # Create predictions using simple model - decision tree
    y_pred = prediction(X_test, clf_object=clf_gini)
    
    # show results
    cal_accuracy(y_test, y_pred)

In [43]:
# Import the SVC model from sklearn
from sklearn.svm import SVC

def SVM_model_tiral(X_train, y_train, X_test, y_test):
    ''' This function tries a Support Vector Classifier to see if we can handle the data volume'''
    # we will try it out on 3 small subsets and compare to simple decision tree.
    svm_trial = SVC(random_state = RSEED)

    # TODO: Calculate the number of samples for 1%, 5%, and 10% of the training data
    samples_1 = int(round(len(X_train) / 100))
    samples_5 = int(round(len(X_train) / 20))
    samples_10 = int(round(len(X_train) / 10))

    # Collect results on the learners
    results = {}
    for clf in [svm_trial]:
        clf_name = svm_trial.__class__.__name__
        results[clf_name] = {}
        for i, samples in enumerate([samples_1, samples_5, samples_10]):
            results[clf_name][i] = \
            train_predict(clf, samples, X_train, y_train, X_test, y_test)
    for i in results.items():
        print (i[0])
        display(pd.DataFrame(i[1]).rename(columns={0:'1%', 1:'5%', 2:'10%'}))
# Results show that it is already very slow on 1% and 5% of data
# and the results are not promising so we will try other models

In [20]:
# hashed out because it takes so long
#SVM_model_tiral(X_train, y_train, X_test, y_test)


In [25]:
# Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

def complex_model_selection(X_train, y_train, X_test, y_test):
    '''This function runs three models to determine which model is the best candidate for further review'''
    # Initialize the three models, the random states are set to RSEED so we know how to reproduce the model later
    clf_A = DecisionTreeClassifier(random_state=RSEED)
    clf_B = RandomForestClassifier(random_state = RSEED, n_jobs=-1)
    clf_C = AdaBoostClassifier(random_state = RSEED)

    # Calculate the number of samples for 1%, 10%, and 100% of the training data
    samples_1 = int(round(len(X_train) / 100))
    samples_10 = int(round(len(X_train) / 10))
    samples_100 = int(round(len(X_train) / 1))

    # Collect results on the learners
    results = {}
    for clf in [clf_A, clf_B, clf_C]:
        clf_name = clf.__class__.__name__
        results[clf_name] = {}
        for i, samples in enumerate([samples_1, samples_10, samples_100]):
            results[clf_name][i] = \
            train_predict(clf, samples, X_train, y_train, X_test, y_test)
        
    # Printing out the values
    for i in results.items():
        print (i[0])
        display(pd.DataFrame(i[1]).rename(columns={0:'1%', 1:'10%', 2:'100%'}))

In [26]:
complex_model_selection(X_train, y_train, X_test, y_test)

DecisionTreeClassifier trained on 1615 samples.
DecisionTreeClassifier trained on 16153 samples.
DecisionTreeClassifier trained on 161528 samples.
RandomForestClassifier trained on 1615 samples.
RandomForestClassifier trained on 16153 samples.
RandomForestClassifier trained on 161528 samples.
AdaBoostClassifier trained on 1615 samples.
AdaBoostClassifier trained on 16153 samples.
AdaBoostClassifier trained on 161528 samples.
DecisionTreeClassifier


,1%,10%,100%
train_time,0.033539,0.441925,7.688792
pred_time,0.147962,0.057216,0.060279
acc_train,1.000000,0.996667,0.986667
acc_test,0.641532,0.689597,0.751306
f_train,1.000000,0.998865,0.988701
f_test,0.690084,0.732305,0.781459


RandomForestClassifier


,1%,10%,100%
train_time,0.198509,0.865281,17.196880
pred_time,0.217174,0.273195,0.530815
acc_train,1.000000,1.000000,0.983333
acc_test,0.714013,0.742268,0.789565
f_train,1.000000,1.000000,0.981048
f_test,0.750985,0.776522,0.813041


AdaBoostClassifier


,1%,10%,100%
train_time,0.159644,1.252806,15.000945
pred_time,0.782230,0.791573,0.794514
acc_train,0.803333,0.776667,0.730000
acc_test,0.714855,0.729589,0.737043
f_train,0.825190,0.801938,0.764263
f_test,0.751933,0.762300,0.772646


In [ ]:
'''df['success'] = df.state.apply(lambda x: True if x == 'successful' else False)
df['match'] = df['success'] == df['reached_goal']
df.query('state == "successful"').match.value_counts()'''

In [ ]:
# Stuff for figureing out exchange rates
'''df['implied_fx_rate'] = df['usd_pledged'] / df['pledged']
df[['usd_pledged','implied_fx_rate','fx_rate','static_usd_rate']].head(10)
df['usd_goal'] = df['goal'] * df['static_usd_rate']
df['reached_goal'] = df['usd_goal'] < df['usd_pledged']
df['success'] = df.state.apply(lambda x: True if x == 'successful' else False)
df['match'] = df['success'] == df['reached_goal']
df.query('state == "successful" & usd_type == "international"').match.value_counts()'''